<a href="https://colab.research.google.com/github/Adrielcuesta/tesismma/blob/main/Notebook_Colab_RAG_para_An%C3%A1lisis_de_Riesgos_(Versi%C3%B3n_1_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################
#                    CUADERNO DE GOOGLE COLAB: RAG PARA ANÁLISIS DE RIESGOS (V1.3)
#                                       Revisado y Comentado
################################################################################

#-------------------------------------------------------------------------------
# PASO 0: CONFIGURACIÓN INICIAL Y LIBRERÍAS
#-------------------------------------------------------------------------------
# Markdown:
# Esta celda se encarga de instalar todas las bibliotecas de Python necesarias para
# el funcionamiento de la notebook. Utilizamos `!pip install -q -U` para una instalación
# silenciosa y para asegurar que se obtengan versiones actualizadas.
# Las bibliotecas incluyen:
# - `google-generativeai`: Para interactuar con la API de Gemini.
# - `langchain`: El framework principal para construir la aplicación RAG.
# - `langchain-community`: Contiene integraciones comunitarias para LangChain,
#   como ChromaDB y SentenceTransformerEmbeddings, que son cruciales.
# - `langchain-google-genai`: Integración específica de LangChain para los modelos de Google (Gemini).
# - `sentence-transformers`: Para generar los embeddings de texto.
# - `chromadb`: Para la base de datos vectorial donde se almacenarán los embeddings.
# - `pymupdf`: Para cargar y extraer texto de documentos PDF.
# Es crucial que esta celda se ejecute primero para asegurar que todas las dependencias
# estén disponibles para el resto del código.

# Instalar las bibliotecas necesarias, asegurando langchain-community
!pip install -q -U google-generativeai langchain langchain-community langchain-google-genai sentence-transformers chromadb pymupdf

import os
import json
import shutil # Para manejar directorios
from datetime import datetime

# LangChain
from langchain_community.vectorstores import Chroma # Importación corregida
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader # Importación corregida
from langchain.chains import RetrievalQA
from langchain_community.embeddings import SentenceTransformerEmbeddings # Importación corregida
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Google Generative AI (Gemini)
import google.generativeai as genai

# Para montar Google Drive
from google.colab import drive
from google.colab import userdata # Para gestionar API keys de forma segura

print("--- Configuración Inicial: Bibliotecas instaladas/actualizadas ---")

#-------------------------------------------------------------------------------
# PASO 1: MONTAR GOOGLE DRIVE Y CONFIGURAR API KEY
#-------------------------------------------------------------------------------
# Markdown:
# En esta sección, realizamos dos acciones fundamentales:
# 1. Montar Google Drive: Esto permite a la notebook acceder a los archivos
#    (documentos PDF, base de datos persistente) almacenados en tu Google Drive.
#    Al ejecutar `drive.mount`, se te pedirá autorización.
# 2. Configurar la API Key de Gemini: Para utilizar los modelos de lenguaje de Google (Gemini),
#    necesitamos una API Key. Esta clave se obtiene de Google AI Studio y se guarda
#    de forma segura utilizando la función "Secretos" de Google Colab.
#    **Acción Requerida por el Usuario:**
#    - Debes haber creado una API Key en Google AI Studio (https://aistudio.google.com/app/apikey).
#    - Debes haber guardado esta API Key en los "Secretos" de Colab con el nombre `GEMINI_API_KEY`.
#      (Panel izquierdo de Colab -> Ícono de llave 🔑 -> Agregar nuevo secreto).

# Montar Google Drive
try:
    drive.mount('/content/drive', force_remount=True) # force_remount puede ayudar si hay problemas de montaje
    print("--- Google Drive montado exitosamente ---")
except Exception as e:
    print(f"Error al montar Google Drive: {e}")
    print("Asegúrate de que tienes permiso para acceder a Google Drive y que la ventana emergente de autorización fue completada.")

# Configurar la API Key de Gemini
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("La API Key de Gemini no se encontró en los Secretos de Colab. Por favor, créala y configúrala.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("--- API Key de Gemini configurada exitosamente ---")
except Exception as e:
    print(f"Error al configurar la API Key de Gemini: {e}")
    print("Por favor, asegúrate de haber guardado tu GEMINI_API_KEY en los Secretos de Colab y que tenga acceso al notebook.")
    GEMINI_API_KEY = None

#-------------------------------------------------------------------------------
# PASO 2: DEFINIR RUTAS Y PARÁMETROS
#-------------------------------------------------------------------------------
# Markdown:
# Aquí definimos todas las rutas a las carpetas en Google Drive que utilizará la notebook,
# así como los parámetros clave para el procesamiento de los documentos y el modelo RAG.
# **Acción Requerida por el Usuario:**
# - Debes **AJUSTAR** la variable `DRIVE_PROJECT_PATH` para que apunte a la carpeta
#   principal que creaste en tu Google Drive para este proyecto.
# Las subcarpetas (`BaseConocimiento`, `ProyectoAnalizar`, `ChromaDB_V1`, `Resultados`)
# se crearán automáticamente si no existen dentro de `DRIVE_PROJECT_PATH`.
# También se definen parámetros como el tamaño de los fragmentos (`CHUNK_SIZE`),
# el solapamiento (`CHUNK_OVERLAP`), el modelo de embeddings, el modelo LLM y
# la cantidad de documentos a recuperar (`K_RETRIEVED_DOCS`).

# Rutas en Google Drive (ajusta según tu estructura de carpetas)
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/Tesis_RAG_Colab/" # ¡¡¡AJUSTA ESTA RUTA SI ES NECESARIO!!!

# Subcarpetas dentro del proyecto
DOCS_BASE_CONOCIMIENTO_PATH = os.path.join(DRIVE_PROJECT_PATH, "BaseConocimiento")
PROYECTO_A_ANALIZAR_PATH = os.path.join(DRIVE_PROJECT_PATH, "ProyectoAnalizar")
CHROMA_DB_PATH = os.path.join(DRIVE_PROJECT_PATH, "ChromaDB_V1")
OUTPUT_PATH = os.path.join(DRIVE_PROJECT_PATH, "Resultados")

# Crear directorios si no existen
os.makedirs(DOCS_BASE_CONOCIMIENTO_PATH, exist_ok=True)
os.makedirs(PROYECTO_A_ANALIZAR_PATH, exist_ok=True)
os.makedirs(CHROMA_DB_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)

print(f"Ruta base del proyecto en Drive: {DRIVE_PROJECT_PATH}")
print(f"  Carpeta Base de Conocimiento: {DOCS_BASE_CONOCIMIENTO_PATH}")
print(f"  Carpeta Proyecto a Analizar: {PROYECTO_A_ANALIZAR_PATH}")
print(f"  Carpeta ChromaDB: {CHROMA_DB_PATH}")
print(f"  Carpeta Resultados: {OUTPUT_PATH}")

# Parámetros de procesamiento
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 150
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
LLM_MODEL_NAME = 'gemini-1.5-flash-latest'
K_RETRIEVED_DOCS = 3

print(f"\n--- Parámetros Definidos ---")
print(f"Chunk Size: {CHUNK_SIZE}, Chunk Overlap: {CHUNK_OVERLAP}")
print(f"Modelo de Embeddings: {EMBEDDING_MODEL_NAME}")
print(f"Modelo LLM: {LLM_MODEL_NAME}")
print(f"Documentos a recuperar (k): {K_RETRIEVED_DOCS}")

#-------------------------------------------------------------------------------
# PASO 3: FUNCIONES DE CARGA Y PROCESAMIENTO DE DOCUMENTOS
#-------------------------------------------------------------------------------
# Markdown:
# Esta celda define la función `cargar_y_procesar_pdfs_de_carpeta`.
# Su propósito es:
# 1. Iterar sobre todos los archivos en una carpeta especificada de Google Drive.
# 2. Si un archivo es un PDF, cargarlo usando `PyMuPDFLoader` (de `langchain_community.document_loaders`).
#    Este cargador trata cada página del PDF como un "Documento" individual en LangChain.
# 3. Añadir metadatos a cada página cargada:
#    - `source_document`: El nombre del archivo PDF original.
#    - `page_number`: El número de la página (ajustado para ser 1-indexed).
# 4. Una vez cargadas todas las páginas de todos los PDFs, fragmentar el texto
#    de estos documentos usando `RecursiveCharacterTextSplitter` con los
#    parámetros `CHUNK_SIZE` y `CHUNK_OVERLAP` definidos anteriormente.
# 5. Devolver una lista de todos los fragmentos generados.
# **Archivos Requeridos:** Esta función espera que los archivos PDF estén presentes
# en la carpeta que se le pase como argumento (ej. `DOCS_BASE_CONOCIMIENTO_PATH`).

def cargar_y_procesar_pdfs_de_carpeta(carpeta_path):
    """
    Carga todos los archivos PDF de una carpeta, los divide en fragmentos
    y añade metadatos básicos.
    """
    documentos_cargados = []
    if not os.path.isdir(carpeta_path):
        print(f"Error: La carpeta especificada no existe: {carpeta_path}")
        return []

    print(f"Procesando PDFs desde la carpeta: {carpeta_path}")
    for filename in os.listdir(carpeta_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(carpeta_path, filename)
            try:
                loader = PyMuPDFLoader(file_path)
                docs_del_pdf = loader.load()

                for doc_page in docs_del_pdf:
                    doc_page.metadata["source_document"] = filename
                    doc_page.metadata["page_number"] = doc_page.metadata.get('page', -1) + 1

                documentos_cargados.extend(docs_del_pdf)
                print(f"  Cargado y procesado preliminarmente: {filename} ({len(docs_del_pdf)} páginas)")
            except Exception as e:
                print(f"  Error al cargar o procesar {filename}: {e}")
        else:
            print(f"  Archivo omitido (no es PDF): {filename}")


    if not documentos_cargados:
        print(f"No se encontraron o no se pudieron cargar PDFs válidos en: {carpeta_path}")
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        add_start_index=True
    )
    fragmentos = text_splitter.split_documents(documentos_cargados)
    print(f"Total de fragmentos generados de la carpeta '{os.path.basename(carpeta_path)}': {len(fragmentos)}")
    return fragmentos

print("\n--- Funciones de Carga y Procesamiento de Documentos Definidas ---")

#-------------------------------------------------------------------------------
# PASO 4: CREAR/CARGAR BASE DE DATOS VECTORIAL (CHROMA DB)
#-------------------------------------------------------------------------------
# Markdown:
# Este paso es crucial para construir la "memoria" de nuestro sistema RAG.
# Aquí se realizan las siguientes acciones:
# 1. Inicializar el Modelo de Embeddings: Se carga el modelo `all-MiniLM-L6-v2`
#    de `SentenceTransformerEmbeddings` (de `langchain_community.embeddings`).
#    Este modelo convertirá los fragmentos de texto en vectores numéricos (embeddings).
#    Se configura para usar la CPU (`device='cpu'`) para asegurar que funcione
#    en el entorno gratuito de Colab y para reservar la GPU (si está disponible) para el LLM.
# 2. Lógica de Creación/Carga de ChromaDB:
#    - Se define una variable `RECREAR_DB`. Si es `True`, cualquier base de datos
#      ChromaDB existente en `CHROMA_DB_PATH` será eliminada y se creará una nueva.
#      Si es `False` (por defecto), intentará cargar una base de datos existente.
#    - **Creación (si no existe o `RECREAR_DB` es `True`):**
#        - Llama a `cargar_y_procesar_pdfs_de_carpeta` para obtener los fragmentos
#          de los documentos en `DOCS_BASE_CONOCIMIENTO_PATH`.
#        - Si se obtienen fragmentos, se utiliza `Chroma.from_documents` para:
#            - Generar embeddings para cada fragmento usando la `embedding_function` definida.
#            - Almacenar estos fragmentos y sus embeddings en ChromaDB.
#            - Persistir (guardar) la base de datos en la carpeta `CHROMA_DB_PATH`
#              en Google Drive.
#    - **Carga (si ya existe y `RECREAR_DB` es `False`):**
#        - Se utiliza `Chroma` para cargar la base de datos persistida desde
#          `CHROMA_DB_PATH`, utilizando la misma `embedding_function`.
# **Archivos Requeridos:**
# - Si se crea la DB: PDFs en la carpeta `DOCS_BASE_CONOCIMIENTO_PATH`.
# - Si se carga la DB: Archivos de ChromaDB (ej. `chroma.sqlite3`) en `CHROMA_DB_PATH`.

# Inicializar el modelo de embeddings
print(f"Inicializando modelo de embeddings: {EMBEDDING_MODEL_NAME}...")
embedding_function = None # Inicializar a None
try:
    embedding_function = SentenceTransformerEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'}
    )
    print("Modelo de embeddings inicializado en CPU.")
except Exception as e:
    print(f"Error crítico al inicializar el modelo de embeddings: {e}")
    print("La ejecución no puede continuar sin el modelo de embeddings.")

# Variable para la base de datos vectorial
vector_db = None

# Lógica para crear la base de datos solo si no existe o si se fuerza la recreación
RECREAR_DB = False # Cambia a True si quieres borrar y recrear la DB

if embedding_function:
    if RECREAR_DB and os.path.exists(CHROMA_DB_PATH):
        print(f"Borrando base de datos ChromaDB existente en: {CHROMA_DB_PATH} porque RECREAR_DB es True.")
        try:
            shutil.rmtree(CHROMA_DB_PATH)
            os.makedirs(CHROMA_DB_PATH, exist_ok=True)
            print(f"Carpeta {CHROMA_DB_PATH} limpiada y recreada.")
        except Exception as e_shutil:
            print(f"Error al intentar borrar la carpeta de ChromaDB: {e_shutil}")

    chroma_db_file_check = os.path.join(CHROMA_DB_PATH, "chroma.sqlite3")

    if not os.path.exists(chroma_db_file_check) or RECREAR_DB:
        print(f"Intentando crear nueva base de datos vectorial en: {CHROMA_DB_PATH}")
        print("Cargando documentos de la base de conocimiento...")
        fragmentos_base_conocimiento = cargar_y_procesar_pdfs_de_carpeta(DOCS_BASE_CONOCIMIENTO_PATH)

        if fragmentos_base_conocimiento:
            print(f"Creando colección en ChromaDB con {len(fragmentos_base_conocimiento)} fragmentos...")
            try:
                vector_db = Chroma.from_documents(
                    documents=fragmentos_base_conocimiento,
                    embedding=embedding_function,
                    persist_directory=CHROMA_DB_PATH
                )
                vector_db.persist()
                print("Base de datos vectorial creada y fragmentos añadidos exitosamente.")
            except Exception as e_chroma_create:
                print(f"Error crítico al crear la base de datos Chroma: {e_chroma_create}")
        else:
            print("No se cargaron fragmentos de la base de conocimiento. La base de datos vectorial no se creará/actualizará.")
    else:
        print(f"Cargando base de datos vectorial existente desde: {CHROMA_DB_PATH}")
        try:
            vector_db = Chroma(
                persist_directory=CHROMA_DB_PATH,
                embedding_function=embedding_function
            )
            if vector_db and hasattr(vector_db, '_collection') and vector_db._collection and vector_db._collection.count() > 0:
                 print(f"Base de datos vectorial cargada. Contiene {vector_db._collection.count()} fragmentos.")
            elif vector_db and hasattr(vector_db, '_collection') and vector_db._collection and vector_db._collection.count() == 0:
                 print("Base de datos vectorial cargada, pero la colección está vacía. Verifica si se procesaron documentos en la creación.")
            else:
                print("Advertencia: Base de datos vectorial cargada, pero parece no haberse inicializado correctamente o estar vacía.")
                print("Si es la primera vez, asegúrate de que haya PDFs en BaseConocimiento y considera RECREAR_DB=True.")
        except Exception as e_chroma_load:
            print(f"Error crítico al cargar la base de datos Chroma existente: {e_chroma_load}")
            print("Si el problema persiste, considera borrar la carpeta ChromaDB y re-ejecutar para crearla de nuevo (RECREAR_DB=True).")
            vector_db = None
else:
    print("Error crítico: El modelo de embeddings no se inicializó. No se puede proceder con la base de datos vectorial.")

#-------------------------------------------------------------------------------
# PASO 5: CONFIGURAR LLM (GEMINI) Y CADENA RAG (RetrievalQA)
#-------------------------------------------------------------------------------
# Markdown:
# En este paso, configuramos el "cerebro" de nuestro sistema RAG y la lógica de recuperación y generación.
# 1. Configurar el LLM (Gemini):
#    - Se inicializa `ChatGoogleGenerativeAI` con el modelo `gemini-1.5-flash-latest`
#      (o el especificado en `LLM_MODEL_NAME`).
#    - Se utiliza la `GEMINI_API_KEY` configurada previamente.
#    - `temperature=0.3` se establece para obtener respuestas más factuales y menos creativas,
#      adecuado para análisis de riesgos.
#    - `convert_system_message_to_human=True` es una configuración a veces necesaria para
#      modelos Gemini que esperan un formato de mensaje específico.
# 2. Crear el Recuperador (Retriever):
#    - A partir de la `vector_db` (ChromaDB) cargada o creada, se crea un `retriever`.
#    - El retriever se configura para buscar los `K_RETRIEVED_DOCS` fragmentos más
#      similares a la consulta del usuario.
# 3. Definir la Plantilla de Prompt:
#    - Se define una cadena `prompt_template_str` que estructura la instrucción
#      que se enviará al LLM. Esta plantilla incluye marcadores de posición para
#      el `{context}` (los fragmentos recuperados) y la `{question}` (la descripción
#      del nuevo proyecto).
#    - Se instruye al LLM para que actúe como un analista de riesgos y formatee
#      su respuesta como un objeto JSON.
#    - Se crea una instancia de `PromptTemplate` a partir de esta cadena.
# 4. Crear la Cadena RetrievalQA:
#    - Se utiliza `RetrievalQA.from_chain_type` para ensamblar la cadena RAG.
#    - `llm`: El modelo Gemini configurado.
#    - `chain_type="stuff"`: Este método simplemente "amontona" todos los fragmentos
#      recuperados en el prompt. Es el más simple para empezar.
#    - `retriever`: El recuperador de ChromaDB.
#    - `return_source_documents=True`: Para que la cadena devuelva también los
#      fragmentos fuente que utilizó el LLM.
#    - `chain_type_kwargs={"prompt": prompt}`: Se pasa la instancia de `PromptTemplate`
#      personalizada.
# **Requisitos:**
# - `GEMINI_API_KEY` debe estar configurada y ser válida.
# - `vector_db` debe haberse cargado o creado exitosamente en el PASO 4.
# - `embedding_function` debe haberse inicializado correctamente.

llm = None
qa_chain = None

if GEMINI_API_KEY and vector_db and embedding_function:
    try:
        llm = ChatGoogleGenerativeAI(
            model=LLM_MODEL_NAME,
            google_api_key=GEMINI_API_KEY,
            temperature=0.3,
            convert_system_message_to_human=True
        )
        print(f"--- LLM ({LLM_MODEL_NAME}) configurado exitosamente ---")

        retriever = vector_db.as_retriever(search_kwargs={"k": K_RETRIEVED_DOCS})
        print(f"--- Retriever configurado para obtener {K_RETRIEVED_DOCS} fragmentos ---")

        prompt_template_str = """
        Eres un asistente de IA altamente especializado en la identificación y evaluación de riesgos para proyectos de instalación de maquinaria industrial, basándote en el Project Management Body of Knowledge (PMBOK) y documentación técnica.
        Tu tarea es analizar la descripción del "NUEVO PROYECTO" que se proporciona a continuación. Debes utilizar ÚNICAMENTE la información contenida en el "CONTEXTO PROPORCIONADO" (extractos del PMBOK, manuales técnicos, y lecciones de proyectos anteriores) para realizar tu análisis.

        CONTEXTO PROPORCIONADO:
        {context}

        NUEVO PROYECTO (PREGUNTA DEL USUARIO):
        {question}

        INSTRUCCIONES PARA LA RESPUESTA:
        1. Identifica una lista de posibles riesgos específicos para el "NUEVO PROYECTO".
        2. Para cada riesgo identificado, proporciona:
            a. "descripcion_riesgo": Una descripción clara y concisa del riesgo.
            b. "explicacion_riesgo": Una breve explicación de por qué es un riesgo, citando específicamente la(s) parte(s) del "CONTEXTO PROPORCIONADO" que lo sustentan (ej., "Basado en la sección X del Contexto Y..."). Si el contexto no sustenta directamente un riesgo pero la descripción del proyecto lo sugiere, indícalo.
            c. "impacto_estimado": Una estimación del impacto potencial del riesgo (opciones: "Bajo", "Medio", "Alto").
            d. "probabilidad_estimada": Una estimación de la probabilidad de ocurrencia del riesgo (opciones: "Baja", "Media", "Alta").
        3. Si no encuentras información relevante en el "CONTEXTO PROPORCIONADO" para identificar riesgos o responder sobre un aspecto específico del "NUEVO PROYECTO", debes indicarlo claramente diciendo: "No se encontró información suficiente en el contexto proporcionado para evaluar [aspecto específico]". No inventes información.
        4. Formatea TODA tu respuesta como un ÚNICO objeto JSON. El objeto JSON debe tener una clave principal llamada "riesgos_identificados", que contenga una lista de objetos, donde cada objeto representa un riesgo individual con las claves "descripcion_riesgo", "explicacion_riesgo", "impacto_estimado", y "probabilidad_estimada".

        Ejemplo de formato de un riesgo dentro de la lista:
        {{
          "descripcion_riesgo": "Falla en la integración del nuevo sistema de control con la infraestructura existente.",
          "explicacion_riesgo": "El Contexto 3 menciona problemas de compatibilidad con sistemas heredados similares. El Nuevo Proyecto implica una integración con un sistema antiguo.",
          "impacto_estimado": "Alto",
          "probabilidad_estimada": "Media"
        }}

        Comienza tu respuesta JSON:
        """
        prompt = PromptTemplate(
            template=prompt_template_str,
            input_variables=["context", "question"]
        )

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt}
        )
        print("--- Cadena RetrievalQA creada exitosamente ---")

    except Exception as e:
        print(f"Error al configurar el LLM o la cadena RAG: {e}")
        llm = None
        qa_chain = None
else:
    if not GEMINI_API_KEY:
        print("ERROR CRÍTICO: API Key de Gemini no configurada.")
    if not vector_db:
        print("ERROR CRÍTICO: Base de datos vectorial no disponible.")
    if not embedding_function:
        print("ERROR CRÍTICO: Función de embeddings no disponible.")
    print("No se puede continuar con la configuración del LLM y la cadena RAG debido a errores previos.")


#-------------------------------------------------------------------------------
# PASO 6: PROCESAR "PROYECTO A ANALIZAR" Y EJECUTAR CONSULTA
#-------------------------------------------------------------------------------
# Markdown:
# Este es el núcleo del análisis. Aquí se realizan las siguientes acciones:
# 1. Definir el Nombre del PDF a Analizar: Se especifica el nombre del archivo
#    `Proyecto_a_analizar_v1.pdf` que debe estar en la carpeta `PROYECTO_A_ANALIZAR_PATH`.
# 2. Verificar Existencia del Archivo: Se comprueba si el archivo PDF del proyecto existe.
# 3. Procesar el PDF del Proyecto:
#    - Si existe, se carga usando `PyMuPDFLoader`.
#    - Se añaden metadatos (`source_document`, `page_number`).
#    - Se fragmenta el texto usando `RecursiveCharacterTextSplitter` con los mismos
#      parámetros que la base de conocimiento.
# 4. Formular la Consulta: El contenido concatenado de todos los fragmentos del
#    PDF del proyecto se utiliza como la "descripción del nuevo proyecto" o "pregunta"
#    para la cadena RAG. Se incluye un truncamiento si la descripción es demasiado larga
#    para la API de Gemini.
# 5. Ejecutar la Cadena RAG:
#    - Se invoca `qa_chain.invoke()` pasando la descripción del nuevo proyecto.
#    - Se obtiene el resultado del LLM (`resultado_analisis`) y los documentos
#      fuente recuperados por el retriever.
# 6. Imprimir Resultados Crudos: Se muestra la respuesta directa del LLM y la lista
#    de fuentes recuperadas.
# **Archivos Requeridos:**
# - El archivo `Proyecto_a_analizar_v1.pdf` debe estar en la carpeta
#   `PROYECTO_A_ANALIZAR_PATH` definida en el PASO 2.
# **Requisitos:**
# - La `qa_chain` debe haberse inicializado correctamente en el PASO 5.

NOMBRE_PDF_PROYECTO_ANALIZAR = "Proyecto_a_analizar_v1.pdf"
ruta_completa_proyecto_analizar = os.path.join(PROYECTO_A_ANALIZAR_PATH, NOMBRE_PDF_PROYECTO_ANALIZAR)
resultado_analisis = None
fuentes_recuperadas = []

if qa_chain:
    if os.path.exists(ruta_completa_proyecto_analizar):
        print(f"\n--- Procesando el documento para análisis: {NOMBRE_PDF_PROYECTO_ANALIZAR} ---")
        try:
            loader_proyecto = PyMuPDFLoader(ruta_completa_proyecto_analizar)
            doc_proyecto_raw = loader_proyecto.load()

            for page_doc in doc_proyecto_raw:
                page_doc.metadata["source_document"] = NOMBRE_PDF_PROYECTO_ANALIZAR
                page_doc.metadata["page_number"] = page_doc.metadata.get('page', -1) + 1

            text_splitter_proyecto = RecursiveCharacterTextSplitter(
                chunk_size=CHUNK_SIZE,
                chunk_overlap=CHUNK_OVERLAP,
                length_function=len,
                add_start_index=True
            )
            fragmentos_proyecto = text_splitter_proyecto.split_documents(doc_proyecto_raw)

            if not fragmentos_proyecto:
                print(f"Advertencia: No se pudieron extraer fragmentos de {NOMBRE_PDF_PROYECTO_ANALIZAR}. El análisis podría no ser efectivo.")
                # Crear una descripción mínima si no hay fragmentos para evitar error en invoke
                descripcion_nuevo_proyecto = f"Análisis del proyecto contenido en el archivo {NOMBRE_PDF_PROYECTO_ANALIZAR} (no se pudo extraer contenido detallado para la consulta)."
            else:
                descripcion_nuevo_proyecto = "\n\n".join([fp.page_content for fp in fragmentos_proyecto])
                print(f"Descripción del nuevo proyecto generada a partir de {len(fragmentos_proyecto)} fragmentos.")
                # Gemini 1.5 Flash tiene un límite de contexto grande, pero es bueno ser consciente.
                # Un límite práctico para la entrada de la API podría ser alrededor de 30k tokens.
                # 1 token ~= 4 caracteres. 180000 caracteres ~= 45000 tokens. Podría ser mucho.
                # Se reduce el truncamiento para ser más conservador y evitar errores de API.
                MAX_CHARS_PROYECTO = 32000 # Aproximadamente 8k tokens, seguro para Flash
                if len(descripcion_nuevo_proyecto) > MAX_CHARS_PROYECTO:
                    print(f"Advertencia: La descripción del proyecto es muy larga ({len(descripcion_nuevo_proyecto)} caracteres). Se truncará a {MAX_CHARS_PROYECTO} caracteres para la API.")
                    descripcion_nuevo_proyecto = descripcion_nuevo_proyecto[:MAX_CHARS_PROYECTO]

            print("\n--- Ejecutando análisis de riesgos con la cadena RAG... ---")
            if not descripcion_nuevo_proyecto.strip():
                 print("Error: La descripción del proyecto a analizar está vacía. No se puede ejecutar la cadena RAG.")
            else:
                try:
                    respuesta_rag = qa_chain.invoke({"query": descripcion_nuevo_proyecto})
                    resultado_analisis = respuesta_rag.get("result", "No se obtuvo resultado del LLM.")
                    fuentes_recuperadas = respuesta_rag.get("source_documents", [])

                    print("\n--- ANÁLISIS DE RIESGOS GENERADO (RESPUESTA CRUDA DEL LLM) ---")
                    print(resultado_analisis)

                    print("\n--- FUENTES RECUPERADAS POR EL RETRIEVER PARA ESTE ANÁLISIS ---")
                    if fuentes_recuperadas:
                        for i, doc_fuente in enumerate(fuentes_recuperadas):
                            print(f"  Fuente {i+1}: {doc_fuente.metadata.get('source_document', 'N/A')} (Pág: {doc_fuente.metadata.get('page_number', 'N/A')}, Inicio Chunk: {doc_fuente.metadata.get('start_index','N/A')})")
                    else:
                        print("  No se recuperaron fuentes específicas para este análisis.")

                except Exception as e_invoke:
                    print(f"Error crítico durante la invocación de la cadena RAG: {e_invoke}")
                    resultado_analisis = f"Error en análisis: {e_invoke}"

        except Exception as e_proc_proyecto:
            print(f"Error crítico al procesar el PDF del proyecto a analizar '{NOMBRE_PDF_PROYECTO_ANALIZAR}': {e_proc_proyecto}")
            resultado_analisis = f"Error procesando PDF de análisis: {e_proc_proyecto}"
    else:
        print(f"ERROR CRÍTICO: No se encontró el archivo '{NOMBRE_PDF_PROYECTO_ANALIZAR}' en la ruta '{PROYECTO_A_ANALIZAR_PATH}'.")
        print(f"Por favor, sube el archivo a esa ubicación en tu Google Drive ({PROYECTO_A_ANALIZAR_PATH}) y verifica la ruta en el PASO 2.")
else:
    print("ERROR CRÍTICO: La cadena RAG (qa_chain) no está inicializada. Revisa los errores en los pasos anteriores (API Key, Base de Datos Vectorial, Configuración LLM).")

#-------------------------------------------------------------------------------
# PASO 7: PROCESAR SALIDA Y ASIGNAR ESTADO RAG (BÁSICO)
#-------------------------------------------------------------------------------
# Markdown:
# Esta celda toma la respuesta cruda del LLM (`resultado_analisis`) e intenta
# estructurarla.
# 1. Función `intentar_parsear_json_riesgos`:
#    - Intenta extraer un objeto JSON de la respuesta del LLM. El prompt
#      le pide al LLM que formatee la salida como JSON.
#    - Busca el primer `{` y el último `}` para delimitar el JSON de forma más robusta.
# 2. Función `asignar_estado_rag`:
#    - Asigna un color RAG (Rojo, Ámbar, Verde, Gris) basado en las cadenas de
#      texto para impacto y probabilidad que se espera que el LLM proporcione.
#      Esta es una lógica simplificada.
# 3. Procesamiento Principal:
#    - Si `resultado_analisis` es un string, llama a `intentar_parsear_json_riesgos`.
#    - Si el parseo es exitoso y se encuentra la clave `riesgos_identificados`,
#      itera sobre la lista de riesgos, extrae los campos esperados
#      (`descripcion_riesgo`, `explicacion_riesgo`, `impacto_estimado`, `probabilidad_estimada`)
#      y asigna un estado RAG.
#    - La estructura del JSON guardado ahora incluye `timestamp_analisis`,
#      `nombre_proyecto_analizado`, `modelo_llm_usado`, `respuesta_cruda_llm`,
#      `riesgos_identificados_estructurados`, y `fuentes_contexto_recuperadas`.
# 4. Guardar Resultados:
#    - Los resultados se guardan en un archivo JSON en la carpeta `OUTPUT_PATH`
#      de Google Drive. El nombre del archivo incluye el nombre del proyecto analizado
#      y una marca de tiempo.
# **Salida Esperada:** Un archivo JSON en la carpeta `Resultados` de Google Drive.

def intentar_parsear_json_riesgos(texto_llm_str):
    """Intenta extraer un objeto JSON de la respuesta del LLM."""
    if not isinstance(texto_llm_str, str):
        print("Error de parseo: la entrada para `intentar_parsear_json_riesgos` no es un string.")
        return None
    try:
        json_block_start = texto_llm_str.find('{')
        json_block_end = texto_llm_str.rfind('}')

        if json_block_start != -1 and json_block_end != -1 and json_block_end > json_block_start:
            json_candidate_str = texto_llm_str[json_block_start : json_block_end + 1]
            parsed_json = json.loads(json_candidate_str)
            return parsed_json
        else:
            print("Advertencia de parseo: No se encontró una estructura JSON clara ({...}) en la respuesta del LLM.")
            return None
    except json.JSONDecodeError as e:
        print(f"Error de parseo: No se pudo decodificar JSON de la respuesta del LLM. Error: {e}")
        print(f"Respuesta del LLM que causó el error (primeros 500 caracteres):\n{texto_llm_str[:500]}")
        return None
    except Exception as e_gen: # Captura otras posibles excepciones durante el parseo
        print(f"Error de parseo: Error inesperado al intentar parsear JSON: {e_gen}")
        return None

def asignar_estado_rag(impacto_str, probabilidad_str):
    """Asigna un estado RAG basado en impacto y probabilidad (simplificado)."""
    impacto = impacto_str.lower() if isinstance(impacto_str, str) else "desconocido"
    probabilidad = probabilidad_str.lower() if isinstance(probabilidad_str, str) else "desconocido"

    if impacto == "alto":
        if probabilidad in ["media", "alta"]: return "Rojo"
        else: return "Ámbar"
    elif impacto == "medio":
        if probabilidad == "alta": return "Rojo"
        elif probabilidad == "media": return "Ámbar"
        else: return "Verde"
    elif impacto == "bajo":
        if probabilidad == "alta": return "Ámbar"
        else: return "Verde"
    return "Gris (Indeterminado)"


if resultado_analisis:
    print("\n--- PROCESANDO SALIDA DEL LLM PARA FORMATO ESTRUCTURADO ---")
    riesgos_finales_para_guardar = {
        "timestamp_analisis": datetime.now().isoformat(),
        "nombre_proyecto_analizado": NOMBRE_PDF_PROYECTO_ANALIZAR,
        "modelo_llm_usado": LLM_MODEL_NAME,
        "respuesta_cruda_llm": resultado_analisis if isinstance(resultado_analisis, str) else str(resultado_analisis),
        "riesgos_identificados_estructurados": [],
        "fuentes_contexto_recuperadas": []
    }

    if fuentes_recuperadas: # Asegurarse que fuentes_recuperadas no sea None
        for fuente_doc in fuentes_recuperadas:
            if hasattr(fuente_doc, 'metadata') and hasattr(fuente_doc, 'page_content'): # Verificar atributos
                riesgos_finales_para_guardar["fuentes_contexto_recuperadas"].append({
                    "documento_fuente": fuente_doc.metadata.get('source_document', 'N/A'),
                    "pagina": fuente_doc.metadata.get('page_number', 'N/A'),
                    "contenido_fragmento_relevante (primeros 200 chars)": fuente_doc.page_content[:200] + "..."
                })
            else:
                print(f"Advertencia: Documento fuente recuperado con estructura inesperada: {fuente_doc}")


    if isinstance(resultado_analisis, str):
        json_output_llm = intentar_parsear_json_riesgos(resultado_analisis)

        if json_output_llm and "riesgos_identificados" in json_output_llm and isinstance(json_output_llm["riesgos_identificados"], list):
            print("JSON de riesgos parseado exitosamente desde la respuesta del LLM.")
            for riesgo_item_llm in json_output_llm["riesgos_identificados"]:
                if isinstance(riesgo_item_llm, dict):
                    descripcion = riesgo_item_llm.get("descripcion_riesgo", riesgo_item_llm.get("riesgo", "Descripción no proporcionada"))
                    explicacion = riesgo_item_llm.get("explicacion_riesgo", "Explicación no proporcionada")
                    impacto = riesgo_item_llm.get("impacto_estimado", "Desconocido")
                    probabilidad = riesgo_item_llm.get("probabilidad_estimada", "Desconocido")
                    estado_rag = asignar_estado_rag(impacto, probabilidad)

                    riesgos_finales_para_guardar["riesgos_identificados_estructurados"].append({
                        "descripcion_riesgo": descripcion,
                        "explicacion_riesgo_llm": explicacion,
                        "impacto_estimado_llm": impacto,
                        "probabilidad_estimada_llm": probabilidad,
                        "estado_RAG_sugerido": estado_rag
                    })
                else:
                     riesgos_finales_para_guardar["riesgos_identificados_estructurados"].append(
                         {"error_parseo_item_riesgo": "Item no es un diccionario", "contenido_item": str(riesgo_item_llm)}
                     )
            if not riesgos_finales_para_guardar["riesgos_identificados_estructurados"] and json_output_llm["riesgos_identificados"]:
                 print("Advertencia: La lista 'riesgos_identificados' del LLM contenía elementos, pero no eran diccionarios válidos.")
            elif not json_output_llm["riesgos_identificados"]:
                 print("Información: La lista 'riesgos_identificados' del LLM estaba vacía.")

        else:
            print("Advertencia: No se pudo parsear un JSON estructurado de riesgos desde la respuesta del LLM, o la estructura esperada ('riesgos_identificados') no se encontró.")
    else:
        print("Advertencia: El resultado del análisis no es un string, no se intentará parseo JSON. Se guardará la respuesta cruda.")

    timestamp_file = datetime.now().strftime("%Y%m%d_%H%M%S")
    # Limpiar el nombre del archivo PDF para usarlo en el nombre del archivo de salida
    clean_project_name = "".join(c if c.isalnum() else "_" for c in NOMBRE_PDF_PROYECTO_ANALIZAR.replace('.pdf',''))
    output_filename = f"analisis_riesgos_{clean_project_name}_{timestamp_file}.json"
    output_file_path = os.path.join(OUTPUT_PATH, output_filename)

    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(riesgos_finales_para_guardar, f, ensure_ascii=False, indent=4)
        print(f"\n--- Resultado completo del análisis guardado en: {output_file_path} ---")
    except Exception as e_save:
        print(f"Error crítico al guardar el archivo JSON de resultados: {e_save}")

else:
    print("\nNo se generó ningún resultado de análisis en esta ejecución o `resultado_analisis` es None.")

print("\n######################################################################")
print("# FIN DEL SCRIPT DE ANÁLISIS DE RIESGOS RAG (V1.3)                 #")
print("######################################################################")
print("\nRECOMENDACIONES PARA LA EJECUCIÓN:")
print("1. ¡ASEGÚRATE DE HABER AJUSTADO LA RUTA 'DRIVE_PROJECT_PATH' EN EL PASO 2 A TU CARPETA EN GOOGLE DRIVE!")
print("2. Verifica que los documentos PDF de la base de conocimiento estén en la subcarpeta 'BaseConocimiento'.")
print("3. Confirma que el PDF 'Proyecto_a_analizar_v1.pdf' esté en la subcarpeta 'ProyectoAnalizar'.")
print("4. Asegúrate de haber configurado tu 'GEMINI_API_KEY' en los Secretos de Colab (ver instrucciones en PASO 1).")
print("5. Ejecuta todas las celdas en orden secuencial.")
print("6. Revisa los archivos JSON generados en la subcarpeta 'Resultados' de tu carpeta de proyecto en Drive.")
print("7. Si es la primera ejecución o cambiaste los documentos de la base de conocimiento, considera poner RECREAR_DB = True en el PASO 4 y luego vuelve a False para ejecuciones posteriores.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 601.3 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:

<ipython-input-1-77315f800bb7>:235: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Modelo de embeddings inicializado en CPU.
Intentando crear nueva base de datos vectorial en: /content/drive/MyDrive/Tesis_RAG_Colab/ChromaDB_V1
Cargando documentos de la base de conocimiento...
Procesando PDFs desde la carpeta: /content/drive/MyDrive/Tesis_RAG_Colab/BaseConocimiento
  Cargado y procesado preliminarmente: Guía Detallada y Ampliada de Gestión de Riesgos para Proyectos.pdf (25 páginas)
  Cargado y procesado preliminarmente: documento_29.pdf (26 páginas)
  Cargado y procesado preliminarmente: study-of-the-application-of-risk-management-in-the-operation-and-maintenance-of-power-plant-projec.pdf (13 páginas)
Total de fragmentos generados de la carpeta 'BaseConocimiento': 294
Creando colección en ChromaDB con 294 fragmentos...


<ipython-input-1-77315f800bb7>:275: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


Base de datos vectorial creada y fragmentos añadidos exitosamente.
--- LLM (gemini-1.5-flash-latest) configurado exitosamente ---
--- Retriever configurado para obtener 3 fragmentos ---
--- Cadena RetrievalQA creada exitosamente ---

--- Procesando el documento para análisis: Proyecto_a_analizar_v1.pdf ---
Descripción del nuevo proyecto generada a partir de 76 fragmentos.
Advertencia: La descripción del proyecto es muy larga (61604 caracteres). Se truncará a 32000 caracteres para la API.

--- Ejecutando análisis de riesgos con la cadena RAG... ---


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



--- ANÁLISIS DE RIESGOS GENERADO (RESPUESTA CRUDA DEL LLM) ---
```json
{
  "riesgos_identificados": [
    {
      "descripcion_riesgo": "Retrasos en la entrega de materiales eléctricos.",
      "explicacion_riesgo": "El documento menciona la necesidad de materiales eléctricos de primera marca (Schneider, ABB, Siemens, General Electric o similar) y la posibilidad de que la Inspección rechace marcas que no cumplan especificaciones o no estén acreditadas.  Retrasos en la adquisición o entrega de estos materiales podrían retrasar el proyecto.",
      "impacto_estimado": "Medio",
      "probabilidad_estimada": "Media"
    },
    {
      "descripcion_riesgo": "Incumplimiento de las normas de seguridad e higiene.",
      "explicacion_riesgo": "El documento enfatiza la necesidad de cumplir con las normas de seguridad e higiene, incluyendo la presencia de técnicos de Seguridad e Higiene y la presentación de un plan de izaje y montaje aprobado por las autoridades correspondientes. El incumplimi